In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mdata = pd.read_csv('data.csv') #Three stocks (R,X_s,X_b) Without predictors
mdata = np.array(mdata[['r','xs','xb']])

In [12]:
#hyperparams

series_length = 15
total_series_length = 241-series_length
batch_size = 1
truncated_backprop_length = series_length//batch_size
state_size = 4
num_classes = 10
echo_step = 3
num_batches = total_series_length//batch_size//truncated_backprop_length
num_stocks = 3
print(total_series_length//batch_size)
gamma = 1 
train_data = 241 # also an expanding window
jList = []
TWlistTrain = []
TWlist = []
Index = []
MWeights = []
epsilon = 0.1
n = mdata.size/4-4
periods = 60
epochs = 1


226


In [4]:
def generateData():
    begin_idx = randint(0,total_series_length)
    x = mdata.transpose()
    x = x[0:3,begin_idx:begin_idx+series_length]
    y = np.roll(x, echo_step, axis=1)
    y[:,0:echo_step] = 0 
    
    x = x.reshape((num_stocks, batch_size, -1))  
    y = y.reshape((num_stocks, batch_size, -1))
    return (x, y)

data = generateData()

In [5]:
#Initialize Recurrent Neural Network and set-up the placeholders
tf.reset_default_graph()
NN_input = tf.placeholder(tf.float32, [num_stocks, batch_size, truncated_backprop_length])
init_state = tf.placeholder(tf.float32, [num_stocks, batch_size, state_size])
Q_Next = tf.placeholder(tf.float32, [truncated_backprop_length, batch_size, num_classes])

# Weights and biases
W = tf.Variable(np.random.rand(num_stocks, state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((num_stocks,1,state_size)), dtype=tf.float32)
W2 = tf.Variable(np.random.rand(num_stocks, state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((num_stocks, 1,num_classes)), dtype=tf.float32)

inputs_series = tf.unstack(NN_input, axis=2)
labels_series = tf.unstack(Q_Next, axis=0)

#Forward pass
current_state = init_state
states_series = []

for current_input in inputs_series:
    current_input = tf.reshape(current_input, [num_stocks,batch_size, 1])
    input_and_state_concatenated = tf.concat([current_input, current_state],axis=2)  # Increasing number of columns
    next_state = tf.tanh(tf.matmul(input_and_state_concatenated,W) + b)  # Broadcasted addition
    states_series.append(next_state)
    current_state = next_state

#calculate loss
Q_FA = [tf.matmul(state, W2) + b2 for state in states_series]
Q_FA = tf.reduce_sum(Q_FA,axis=1)
A_Max = tf.argmax(Q_FA[-1],1) # only use the latest Q of the RNN for the determination of the optimal weights
Q_series = tf.unstack(Q_FA, axis=0)

# Calculate loss for the NN from the Q values
losses = [ abs(logits - labels) for logits, labels in zip(Q_series,labels_series)]
total_loss = tf.reduce_mean(losses)
# loss = tf.reduce_sum(tf.square(Q_Next - Q_FA))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(total_loss)

#Define Action Matrix (Now discrete case) 
A = np.linspace(0,1,10) # portfolio weights of stocks (1-weight) is the weight in the bonds

In [22]:
with tf.Session() as sess:
    for i in range(241,int(n-periods-1)):
        OptimalWeights = np.zeros(periods-1)
        currentK = 0;
        print(i)
        sess.run(tf.global_variables_initializer()) # initialize the Neural Network again
        
        while currentK < periods - 1:
            #Initilization
            NN_data = mdata[i-200:i+currentK]  # rolling window
            #initialize an empty hidden state
            _current_state = np.zeros((num_stocks, batch_size, state_size))
            # NN_data = mdata[0:i+currentK]   #Expanding window
            rAll = 0
            currentEpoch = 0
            
            while currentEpoch < epochs:
                #Training of the Q-Network for the data available (with Neural Nets) 
                for j in range(0,len(NN_data)-truncated_backprop_length-1):
                    s = NN_data[j:j+truncated_backprop_length,0:3].reshape(num_stocks,batch_size,truncated_backprop_length)
                    #Choose an action by greedily (with e chance of random action) from the Q-network
                    
                    # INITIALIZE THE PLACEHOLDERS
                    a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s, init_state:_current_state})
                    a = A[a_int-1]  # -1 because the output neurons are labeled 1 till 101 and it will be an index
                    if np.random.rand(1) < epsilon:
                        a = random.choice(A)

                    #Get new state and reward from environment
                    s1 = mdata[i+currentK+j+truncated_backprop_length+1,1:4]
                    r = (a*s1[0] + (1-a)*s1[1]) #reward: this is now the wealth gained from this step, but could be other rewards like utility
                    s1 = NN_data[j+1:j+truncated_backprop_length+1,0:3].reshape(num_stocks,batch_size,truncated_backprop_length)
                    Q = sess.run(Q_FA,feed_dict={NN_input:s1, init_state:_current_state})
        
                    #Obtain maxQ' and set our target value for chosen action.
                    Q1 = np.max(Q[-1])
                    targetQ = allQ
                    targetQ[-1,0,a_int] = r + gamma*Q1

                    #Train the neural network using target and predicted Q values
                    # INITIALIZE THE PLACEHOLDERS
                    _,_current_state,W1 = sess.run([updateModel,current_state,NN_input],feed_dict={NN_input:s,Q_Next:targetQ, init_state:_current_state})
                currentEpoch += 1
        
            # After training now calculate the optimal weights for the K=60 periods to come
            s1 = NN_data[j+1:j+truncated_backprop_length+1,0:3].reshape(num_stocks,batch_size,truncated_backprop_length)
            a_int,allQ = sess.run([A_Max,Q_FA],feed_dict={NN_input:s, init_state:_current_state})
            aOpt = A[a_int-1]
            OptimalWeights[currentK] = aOpt
            currentK += 1
        
        # For insight purposes
        MWeights.append(np.mean(OptimalWeights))
        x = mdata[i+currentK+j+truncated_backprop_length+1,1:4]
        TWlist.append(np.exp(sum(OptimalWeights*x[0] + (1-OptimalWeights)*x[1])))
        Index.append(i)
        
        print('Writing away results')
        df = pd.DataFrame({'index date':Index,'TW':TWlist, 'Mean Weights Xs':MWeights})
        df.to_excel('Results_RNN_g10_Other.xlsx', sheet_name='sheet1', index=False)
    # close session
plt.plot(MWeights)
plt.plot(TWlist)

241
Writing away results
242
Writing away results
243
Writing away results
244
Writing away results
245
Writing away results
246
Writing away results
247
Writing away results
248
Writing away results
249
Writing away results
250
Writing away results
251
Writing away results
252
Writing away results
253
Writing away results
254
Writing away results
255
Writing away results
256
Writing away results
257
Writing away results
258
Writing away results
259
Writing away results
260
Writing away results
261
Writing away results
262
Writing away results
263
Writing away results
264
Writing away results
265
Writing away results
266
Writing away results
267
Writing away results
268
Writing away results
269
Writing away results
270
Writing away results
271
Writing away results
272
Writing away results
273
Writing away results
274
Writing away results
275
Writing away results
276
Writing away results
277
Writing away results
278
Writing away results
279
Writing away results


PermissionError: [Errno 13] Permission denied: 'Results_RNN_g10_Other.xlsx'